In [2]:
pip install -U langchain langchain-community ollama ddgs

  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_community.llms import Ollama
from ddgs import DDGS
import time

# Initializing LLM (Ollama)

llm = Ollama(model="llama3")

# Simple Medical Search

def medical_search(query):
    try:
        time.sleep(1)
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=2))
            if results:
                return results[0]["body"]
    except:
        pass

    # fallback knowledge so system never crashes
    return """
General medical guidance:

For ankle sprains:
- Rest 2–3 days
- Ice 15–20 minutes every 2–3 hours
- Compression bandage
- Elevation
- Mild sprains heal in 1–2 weeks
- Moderate sprains take 3–6 weeks

For fever:
- Drink fluids
- Rest
- Acetaminophen if needed
- See doctor if fever lasts >3 days
"""

In [6]:
# Conversation Memory

conversation_memory = ""

# Medical Chat Function


def medical_chat(user_input, use_memory=False):
    global conversation_memory

    if use_memory:
        history = conversation_memory
    else:
        history = ""

    medical_info = medical_search(user_input)

    prompt = f"""
You are a helpful medical assistant.

Conversation history:
{history}

Medical reference:
{medical_info}

User: {user_input}
Doctor:
"""

    response = llm.invoke(prompt)

    if use_memory:
        conversation_memory += f"\nUser: {user_input}\nDoctor: {response}\n"

    return response


In [7]:
# CASE 1 — NO MEMORY

print("CASE 1 — NO MEMORY")

print("User: How can I treat a sprained ankle?")
print("Doctor:", medical_chat("How can I treat a sprained ankle?", use_memory=False))

print("\nUser: How many days should I rest?")
print("Doctor:", medical_chat("How many days should I rest?", use_memory=False))

CASE 1 — NO MEMORY

User: How can I treat a sprained ankle?
Doctor: Hi there! So you're looking for tips on how to treat a sprained ankle? Well, you've come to the right place!

According to our medical reference from August 11, 2022, a sprained ankle is usually treated with at-home care and some exercises to help get you back on your feet. Here are some steps you can follow:

1. **Rest**: Give your ankle a break and avoid putting weight on it for the first few days. This will help reduce swelling and pain.
2. **Ice**: Apply an ice pack wrapped in a cloth to the affected area for 15-20 minutes, several times a day. This will help reduce inflammation and ease pain.
3. **Elevate**: Elevate your ankle above the level of your heart to reduce swelling.
4. **Compression**: Wrap an elastic bandage or compression wrap around the injured ankle to help reduce swelling.
5. **Exercise**: Once the initial swelling has gone down, start doing some gentle exercises to help strengthen the ankle and imp

In [8]:
''' Note: ## Observation (Case 1 – No Memory)

In Case 1, the assistant treats each question independently.
It does not retain information from previous turns, causing follow-up questions to lose context.

This results in generic responses and a less natural user experience.'''

' Note: ## Observation (Case 1 – No Memory)\n\nIn Case 1, the assistant treats each question independently.\nIt does not retain information from previous turns, causing follow-up questions to lose context.\n\nThis results in generic responses and a less natural user experience.'

In [11]:
# CASE 2 — WITH MEMORY

conversation_memory = ""

print("CASE 2 — WITH MEMORY")

print("User: How can I treat a sprained ankle?")
print("Doctor:", medical_chat("How can I treat a sprained ankle?", use_memory=True))

print("\nUser: How many days should I rest?")
print("Doctor:", medical_chat("How many days should I rest?", use_memory=True))

print("\n--- Observation (Case 2) ---")
print("""
With memory enabled, the assistant understands the follow-up question
and gives context-aware advice related to the ankle sprain.

This demonstrates why conversational memory is critical for realistic
medical AI assistants.
""")

CASE 2 — WITH MEMORY
User: How can I treat a sprained ankle?
Doctor: I'm happy to help! Based on our previous conversation, you were asking how to treat a sprained ankle. As a medical assistant, I'd like to provide some additional information to support your recovery.

According to the doctor's advice and medical references, here are some steps you can take to treat a sprained ankle:

1. **RICE principle**: Follow the RICE method to reduce pain and swelling:
	* Rest: Avoid putting weight on the affected ankle.
	* Ice: Apply ice packs for 15-20 minutes, several times a day.
	* Compression: Use an elastic bandage or compression wrap to help reduce swelling.
	* Elevation: Elevate your ankle above the level of your heart to reduce swelling.
2. **Immobilization**: Use a splint or cast to immobilize the ankle and prevent further injury.
3. **Pain management**: Take over-the-counter pain relievers like acetaminophen or ibuprofen as directed by your doctor.
4. **Physical therapy**: As the ankl